In [93]:
######################## Q1 ########################

from queue import Queue

class Node:
    def __init__(self, level, weight, profit, include):
        self.level = level
        self.weight = weight
        self.profit = profit
        self.include = include

def kp_BFS(W, w, p, include):
    node_num = 0
    max_queue_elem =0
    global maxProfit
    global bestset

    Q = Queue()
    v = Node(0, 0, 0, include)
    node_num += 1
    #print(v.level, v.profit, v.weight, compBound(v, W, w, p), v.include)   # 만들어진 Node(root) 출력 (Debug)
    maxProfit = 0
    Q.put(v)
    max_queue_elem = max(max_queue_elem, Q.qsize())

    while not Q.empty():
        v = Q.get()
        # u 포함
        next_include = v.include.copy()
        next_include[v.level+1] = 1
        u = Node(v.level+1, v.weight+w[v.level+1], v.profit+p[v.level+1], next_include)
        node_num += 1
        #print(u.level, u.profit, u.weight, compBound(u, W, w, p), u.include[1:])    # 만들어진 Node 출력 (Debug)
        
        if u.weight <= W and u.profit > maxProfit:
            bestset = u.include
            maxProfit = u.profit

        bound = compBound(u, W, w, p)
        if bound > maxProfit:
            Q.put(u)
            max_queue_elem = max(max_queue_elem, Q.qsize())

        # u 미포함
        next_include = v.include.copy()
        next_include[v.level+1] = 0
        u = Node(v.level+1, v.weight, v.profit, next_include)
        node_num += 1
        #print(u.level, u.profit, u.weight, compBound(u, W, w, p), u.include[1:])    # 만들어진 Node 출력 (Debug)
        bound = compBound(u, W, w, p)
        if bound > maxProfit:
            Q.put(u)
            max_queue_elem = max(max_queue_elem, Q.qsize())

    return node_num, max_queue_elem
        
        
def compBound(u, W, w, p):
    profit = u.profit
    weight = u.weight
    k = 0

    if weight >= W:
        return 0

    ww = weight
    for kk in range(u.level+1, len(w)+1):
        if kk == len(w) :
            k = kk
            break
        ww += w[kk]
        if ww > W:
            k = kk
            break
    totweight = weight + sum(w[u.level+1:k])
    
    if kk == len(w):    # 현재 level 이후의 모든 원소를 넣어도 W보다 작은 경우
        bound = profit + sum(p[u.level+1:])
    else:
        bound = profit + sum(p[u.level+1:k]) + (W - totweight) * p[k]/w[k]

    
    return int(bound)

In [97]:
n = 4
W = 6
p = [30, 28, 18, 20]
w = [3, 4, 3, 5]

# 첫번째 item의 정보: p[1], w[1]
p = [0] + p
w = [0] + w

# 위에서 p, w에 0번 인덱스를 추가하였기 때문에 n+1크기로 배열을 만듦
include = [0] * (n+1)
maxProfit = 0
bestset = [0] * (n+1)
node_num, max_queue_elem = kp_BFS(W, w, p, include)

print('######################## Q1 ########################')
print(bestset[1:])
print(f'상태공간트리의 총 노드 수: {node_num}')
print(f'queue에 저장되어 있는 데이터의 개수의 최대값: {max_queue_elem}')

######################## Q1 ########################
[1, 0, 1, 0]
상태공간트리의 총 노드 수: 11
queue에 저장되어 있는 데이터의 개수의 최대값: 2


In [109]:
######################## Q2 ########################
from queue import PriorityQueue

class Node:
    def __init__(self, level, weight, profit, bound, include):
        self.level = level
        self.weight = weight
        self.profit = profit
        self.include = include
        self.bound = bound
    
    def __lt__(self, other):
        return self.bound < other.bound

def kp_Best_FS(W, w, p, include):
    global maxProfit
    global bestset

    PQ = PriorityQueue()
    v = Node(0, 0, 0, None, include)
    v.bound = -compBound(v, W, w, p)
    #print(v.level, v.profit, v.weight, v.bound, v.include)   # 만들어진 Node(root) 출력 (Debug)
    maxProfit = 0
    PQ.put(v)

    while not PQ.empty():
        v = PQ.get()    # bound값이 가장 큰 Node 추출

        if -v.bound > maxProfit: # Node가 유망한지 확인, minheap으로 PQ가 구현되어 있으므로 bound값은 음수임
            # u 포함
            next_include = v.include.copy()
            next_include[v.level+1] = 1
            u = Node(v.level+1, v.weight+w[v.level+1], v.profit+p[v.level+1], None, next_include)
            u.bound = -compBound(u, W, w, p)
            #print(u.level, u.profit, u.weight, u.bound, u.include[1:])    # 만들어진 Node 출력 (Debug)

            if u.weight <= W and u.profit > maxProfit:
                bestset = u.include
                maxProfit = u.profit
            if -u.bound > maxProfit:     # u가 유망하면 PQ에 입력
                PQ.put(u)

            # u 미포함
            next_include = v.include.copy()
            next_include[v.level+1] = 0
            u = Node(v.level+1, v.weight, v.profit, None, next_include)
            u.bound = -compBound(u, W, w, p)
            #print(u.level, u.profit, u.weight, u.bound, u.include[1:])    # 만들어진 Node 출력 (Debug)

            if -u.bound > maxProfit:     # u가 유망하면 PQ에 입력
                PQ.put(u)
        
        
def compBound(u, W, w, p):
    profit = u.profit
    weight = u.weight
    k = 0

    if weight >= W:
        return 0

    ww = weight
    for kk in range(u.level+1, len(w)+1):
        if kk == len(w) :
            k = kk
            break
        ww += w[kk]
        if ww > W:
            k = kk
            break
    totweight = weight + sum(w[u.level+1:k])
    
    if kk == len(w):    # 현재 level 이후의 모든 원소를 넣어도 W보다 작은 경우
        bound = profit + sum(p[u.level+1:])
    else:
        bound = profit + sum(p[u.level+1:k]) + (W - totweight) * p[k]/w[k]

    
    return int(bound)

In [111]:
n = 4
W = 6
p = [30, 28, 18, 20]
w = [3, 4, 3, 5]

# 첫번째 item의 정보: p[1], w[1]
p = [0] + p
w = [0] + w

# 위에서 p, w에 0번 인덱스를 추가하였기 때문에 n+1크기로 배열을 만듦
include = [0] * (n+1)
maxProfit = 0
bestset = [0] * (n+1)

kp_Best_FS(W, w, p, include)

print('######################## Q2 ########################')
print(bestset[1:])

######################## Q2 ########################
[1, 0, 1, 0]
